In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [3]:
loader=PyPDFDirectoryLoader("./us_cencus")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

final_documents=text_splitter.split_documents(documents)

final_documents[0]

Document(metadata={'source': 'us_cencus/acsbr-015.pdf', 'page': 0, 'page_label': '1'}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under \nthe Continuous Enrollment Provision.2 The American \nRescue Plan (ARP) enhanced Marketplace premium \nsubsidies for those with incomes above 400 percent \nof the poverty level as well as for unemployed people.3\nIn addition to national policies, individual states and \nthe District of Co

In [4]:
print(len(final_documents))

280


In [9]:
##Embedding using hugging face
huggingface_Embeddings = HuggingFaceBgeEmbeddings(
    model_name = "BAAI/bge-small-en-v1.5",
    model_kwargs={'device':'cpu'},
    encode_kwargs={'noramlize_embeddings':True}
)

/Users/ekaspreetsinghatwal/Desktop/LangChain/hello/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
import numpy as np
arr = np.array(huggingface_Embeddings.embed_query(final_documents[0].page_content))
print(np.shape(arr))

(384,)


In [15]:
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_Embeddings)

In [16]:
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [17]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x30b43e480> search_kwargs={'k': 3}


In [66]:
import os
from langchain_community.llms import HuggingFaceHub
import dotenv

dotenv.load_dotenv()  # Corrected line
# Retrieve HuggingFace API token from environment variable
HUGGINGFACEHUB_API_TOKEN = os.getenv("HF_API_TOKEN")



In [74]:
from langchain_community.llms import HuggingFaceHub

hf = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature": 0.1, "max_length": 500},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN  # Add your token here
)

query = "What is the health insurance coverage?"
hf.invoke(query)


/Users/ekaspreetsinghatwal/Desktop/LangChain/hello/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


"What is the health insurance coverage? What is the health insurance coverage? Health insurance coverage refers to the benefits and services that a health insurance plan provides to its members. This can include:\n\n1. **Medical Services**: This includes coverage for doctor visits, hospitalizations, surgeries, and other medical treatments. The specific services covered can vary depending on the plan.\n\n2. **Prescription Drugs**: Many health insurance plans cover the cost of prescription medications, often with some form of cost-sharing (like copays or coinsurance).\n\n3. **Mental Health Services**: Some plans cover mental health services, such as therapy and counseling sessions, and sometimes medication.\n\n4. **Preventive Care**: Many plans cover preventive services like annual check-ups, immunizations, and screenings at no additional cost.\n\n5. **Emergency Services**: Most plans cover emergency services, although there may be different rules for in-network and out-of-network provid

In [77]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

llm = hf 
llm.invoke(query)

KeyboardInterrupt: 

In [ ]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [ ]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [ ]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [ ]:
result = retrievalQA.invoke({"query": query})
print(result['result'])